In [ ]:
import os
os.chdir('..')
print(os.getcwd())

In [ ]:
import pandas as pd
import json
import copy
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from shapely.geometry import Polygon

## Load

In [ ]:
train = pd.read_csv('data/train.csv')
hubmap = pd.read_csv('data/HuBMAP-20-dataset_information.csv')

In [ ]:
print(train.shape)
train.head(n=3)

In [ ]:
print(hubmap.shape)
hubmap.head(n=3)

## Visualization of train data

In [ ]:
hubmap['id'] = hubmap['image_file'].str.replace('.tiff', '')
train_hubmap = hubmap[hubmap['id'].isin(train['id'].to_list())].reset_index(drop=True)
train_hubmap.head(n=8)

In [ ]:
train_imgs = train_hubmap['id'].to_list()

markups = []
for img in train_imgs:
    json_path = 'data/markup/'
    with open(f'{json_path}{img}.json') as json_file:
        markup = json.load(json_file)
    markups.append(markup)
    
anot_structures = []
for img in train_imgs:
    json_path = 'data/markup/'
    with open(f'{json_path}{img}-anatomical-structure.json') as json_file:
        anot_structure = json.load(json_file)
    anot_structures.append(anot_structure)
    
train_widths = train_hubmap['width_pixels'].to_list()
train_heights = train_hubmap['height_pixels'].to_list()

### 1) Glomerulus_segmentation_file

#### Uniqueness

In [ ]:
types = [glom['type'] for markup in markups for glom in markup]
print(f'Unique features: {list(np.unique(types))}')

ids = [glom['id']  for markup in markups for glom in markup]
print(f'Unique ids: {list(np.unique(ids))}')   

gem_types = [glom['geometry']['type']  for markup in markups for glom in markup]
print(f"Unique geometry types: {list(np.unique(gem_types))}")

prop_names = [glom['properties']['classification']['name']  for markup in markups for glom in markup]
print(f"Unique property names: {list(np.unique(prop_names))}")

prop_islocked = [glom['properties']['isLocked']  for markup in markups for glom in markup]
print(f"Unique isLocked values: {list(np.unique(prop_islocked))}")

rgb = [glom['properties']['classification']['colorRGB']  for markup in markups for glom in markup]
print(f"Unique RGB colours: {list(np.unique(rgb))}")

In [ ]:
unique, counts = np.unique(ids, return_counts=True)
print('Ids distribution:  {}'.format({val: count for val, count in zip(unique, counts)}))

unique, counts = np.unique(prop_islocked, return_counts=True)
print('Islocked distribution:  {}'.format({val: count for val, count in zip(unique, counts)}))

In [ ]:
img_anot = np.unique([img for markup, img in zip(markups, train_imgs) for glom in markup if glom['id'] == 'PathAnnotationObject'])
print(f"Imgs with PathAnnotationObject id: {list(img_anot)}")

img_det = np.unique([img for markup, img in zip(markups, train_imgs) for glom in markup if glom['id'] == 'PathDetectionObject'])
print(f"Imgs with PathDetectionObject id: {list(img_det)}")

#### 2D histogram

In [ ]:
def plot_hist(x: list, y: list=None, subtitles: list=None, bins: int=25, title: str="" ,hist_type='2D',
             x_label: str=None, y_label: str=None):
    """Plot 1D or 2d hist with 4 cols.
    """
    
    if subtitles is None:
        subtitles = [''] * len(x)
    length = len(x)
    num_rows = length // 4
    plt.rcParams['figure.figsize'] = [20, 8]
    fig, ax = plt.subplots(num_rows, 4)
    if hist_type == '2D':
        [ax[i//4, i%4].hist2d(x[i], y[i], bins=bins) for i in range(length)]
        [ax[i//4, i%4].set_xlim([0.0, 1.0]) for i in range(length)]
        [ax[i//4, i%4].set_ylim([0.0, 1.0]) for i in range(length)]
    else:
        [ax[i//4, i%4].hist(x[i], bins=bins) for i in range(length)]
    [ax[i//4, i%4].set_title(subtitles[i], fontsize=16) for i in range(length)]
    fig.text(0.5, 0.04, x_label, ha='center' ,fontsize=24)
    fig.text(0.08, 0.5, y_label, va='center', rotation='vertical', fontsize=24)
    fig.suptitle(title, fontsize=24)
    
def flatten_list(list_of_list: list):
    """Flatten list of list to list.
    """
    return [subsublist for sublist in list_of_list for subsublist in sublist]
    

x - width<br>
y - height

In [ ]:
x = [[x_sample/width for glom in markup for x_sample, y_sample in glom['geometry']['coordinates'][0]] 
     for width, markup in zip(train_widths, markups)]
y = [[y_sample/height for glom in markup for x_sample, y_sample in glom['geometry']['coordinates'][0]] 
     for height, markup in zip(train_heights, markups)]

In [ ]:
###########
bins = 50
###########
plot_hist(x, y, train_imgs, bins=bins, title=f'Glomerulus Segmentation | Normalized | Train | bins={bins}',
         x_label='Width', y_label='Height')

In [ ]:
###########
bins = 100
###########
plt.rcParams['figure.figsize'] = [10, 4]
plt.title(f'Glomerulus Segmentation| Normalized | Train | bins={bins}', fontsize=20)
plt.hist2d(flatten_list(x), flatten_list(y), bins=bins)
plt.xlabel('Width', fontsize=16)
plt.ylabel('Height', fontsize=16); plt.show()

#### Number of Edges

In [ ]:
glom_len = [[len(coords) for glom in markup for coords in glom['geometry']['coordinates']] 
     for markup in markups]

In [ ]:
###########
bins = 100
###########
plot_hist(glom_len, subtitles=train_imgs, bins=bins, title=f'Number of Edges | Glomerulus | Distribution | Train | bins={bins}', 
          hist_type='1D', x_label='Number of edges', y_label='Frequency')

In [ ]:
###########
bins = 100
###########
plt.rcParams['figure.figsize'] = [10, 4]
plt.title(f'Number of edges | Glomerulus | Distribution | Train | bins={bins}', fontsize=24)
plt.hist(flatten_list(glom_len), bins=bins)
plt.ylabel("Frequency", fontsize=20)
plt.xlabel("Number of edges", fontsize=20); plt.show()

#### Area

Assumption: glomerulus has shape of elips<br>
Area: Pi*x_radius*y_radius

In [ ]:
def get_diff_x(coords):
    return list(np.max(coords, axis=0)-np.min(coords, axis=0))[0]
def get_diff_y(coords):
    return list(np.max(coords, axis=0)-np.min(coords, axis=0))[1]

In [ ]:
glom_max_x_diff = [[get_diff_x(coords)/width for glom in markup for coords in glom['geometry']['coordinates']] 
     for width, markup in zip(train_widths, markups)]
glom_max_y_diff = [[get_diff_y(coords)/height for glom in markup for coords in glom['geometry']['coordinates']] 
     for height, markup in zip(train_heights, markups)]
glom_area = [[round(np.pi*x*y*100/4, 3) for x, y in zip( x_img, y_img)] for x_img, y_img in zip(glom_max_x_diff, glom_max_y_diff)] 

In [ ]:
###########
bins = 50
###########
plot_hist(glom_area, subtitles=train_imgs, bins=bins, title=f'Area | Glomerulus | Distribution | Train | bins={bins}', 
          hist_type='1D', x_label='Area, %', y_label='Frequency')

In [ ]:
###########
bins = 100
###########
total_area = np.sum(flatten_list(glom_area))/len(glom_area)
plt.rcParams['figure.figsize'] = [10, 4]
plt.title(f'Area | Glomerulus | Distribution | Train | bins={bins}', fontsize=24)
plt.hist(flatten_list(glom_area), bins=bins)
plt.ylabel("Frequency", fontsize=20)
plt.xlabel("Area, %", fontsize=20); plt.show()

In [ ]:
plt.title("Total Area Filled with Glomerolous | Train ", fontsize=30)
total_areas = [np.sum(glom) for glom in glom_area]
plt.bar(train_imgs, total_areas)
plt.ylabel("Area, %", fontsize=24)
plt.text(-3, int(np.max(total_areas)), f"Mean area {round(np.mean(total_areas), 2)}%", fontsize=12);

### Form

In [ ]:
xy_max = [[np.max(glom['geometry']['coordinates'][0], axis=0) for glom in markup] for markup in markups]
xy_min = [[np.min(glom['geometry']['coordinates'][0], axis=0) for glom in markup] for markup in markups]

In [ ]:
markups_copy = copy.deepcopy(markups)
for i in range(len(markups_copy)):
    for j in range(len(markups_copy[i])):
        for k in range(len(markups_copy[i][j]['geometry']['coordinates'][0])):
            diff_x = xy_max[i][j][0] - xy_min[i][j][0]
            diff_y = xy_max[i][j][1] - xy_min[i][j][1]
            markups_copy[i][j]['geometry']['coordinates'][0][k][0] = (markups_copy[i][j]['geometry']['coordinates'][0][k][0]-xy_min[i][j][0]) / diff_x
            markups_copy[i][j]['geometry']['coordinates'][0][k][1] = (markups_copy[i][j]['geometry']['coordinates'][0][k][1]-xy_min[i][j][1]) / diff_y

In [ ]:
x = [[x_sample for glom in markup for x_sample, y_sample in glom['geometry']['coordinates'][0]] 
     for markup in markups_copy]
y = [[y_sample for glom in markup for x_sample, y_sample in glom['geometry']['coordinates'][0]] 
     for markup in markups_copy]

In [ ]:
###########
bins = 25
###########
plot_hist(x, y, train_imgs, bins=bins, title=f'Glomerulus Form | Normalized | Train | bins={bins}',
         x_label='Width', y_label='Height')

In [ ]:
###########
bins = 50
###########
plt.rcParams['figure.figsize'] = [10, 4]
plt.title(f'Glomerulus Form | Normalized | Train | bins={bins}', fontsize=20)
plt.hist2d(flatten_list(x), flatten_list(y), bins=bins)
plt.xlabel('Width', fontsize=16)
plt.ylabel('Height', fontsize=16); plt.show()

### 2) anatomical_structures_segmention_file

#### Uniqueness

In [ ]:
types = [struct['type'] for anot_structure in anot_structures for struct in anot_structure]
print(f'Unique features: {list(np.unique(types))}')

ids = [struct['id'] for anot_structure in anot_structures for struct in anot_structure]
print(f'Unique ids: {list(np.unique(ids))}')

gem_types = [struct['geometry']['type'] for anot_structure in anot_structures for struct in anot_structure]
print(f'Unique geometry types: {list(np.unique(gem_types))}')


prop_names = [struct['properties']['classification']['name'] for anot_structure in anot_structures for struct in anot_structure]
print(f'Unique property names: {list(np.unique(prop_names))}')

prop_islocked = [struct['properties']['isLocked'] for anot_structure in anot_structures for struct in anot_structure]
print(f'Unique isLocked values: {list(np.unique(prop_islocked))}')

rgb = [struct['properties']['classification']['colorRGB'] for anot_structure in anot_structures for struct in anot_structure]
print(f'Unique RGB colours: {list(np.unique(rgb))}')

In [ ]:
unique, counts = np.unique(gem_types, return_counts=True)
print('Geometry types distribution:  {}'.format({val: count for val, count in zip(unique, counts)}))

unique, counts = np.unique(prop_names, return_counts=True)
print('Property names distribution:  {}'.format({val: count for val, count in zip(unique, counts)}))

unique, counts = np.unique(rgb, return_counts=True)
print('RGB distribution:  {}'.format({val: count for val, count in zip(unique, counts)}))

In [ ]:
print("Example of multipolygon")
multi_pol = [struct['geometry']['coordinates'] for anot_structure in anot_structures for struct in anot_structure
            if struct['geometry']['type'] == 'MultiPolygon']
[print(f"Polygon: {subpol}\n") for pol in multi_pol[0] for subpol in pol];

#### 2D histogram

In [ ]:
def convert_multi_into_plain(coords: list):
    print(coords)
    list_coord = []
    for coord in coords:
        list_coord.append(np.array(coord[0]))
    return np.concatenate(list_coord, axis=0).tolist()

Flatten multi polygon into plain polygon

In [ ]:
%%capture
for i in range(len(anot_structures)):
    for j in range(len(anot_structures[i])):
        if anot_structures[i][j]['geometry']['type'] == 'MultiPolygon':
            anot_structures[i][j]['geometry']['coordinates'] = [convert_multi_into_plain(anot_structures[i][j]['geometry']['coordinates'])] 

In [ ]:
x_anom = [[x_sample/width for struct in anot_structure for x_sample, y_sample in struct['geometry']['coordinates'][0]] 
           for width, anot_structure in zip(train_widths, anot_structures)]
y_anom = [[y_sample/height for struct in anot_structure for x_sample, y_sample in struct['geometry']['coordinates'][0]] 
          for height, anot_structure in zip(train_heights, anot_structures)]

In [ ]:
###########
bins = 50
###########
plot_hist(x_anom, y_anom, train_imgs, bins=bins, title='Anatomical Structures Segmention | Normalized | Train | bins=100',
         x_label='Width', y_label='Height')

#### Area

In [ ]:
an_max_x_diff = [[get_diff_x(coords)/width for struct in anot_structure for coords in struct['geometry']['coordinates']] 
     for width, anot_structure in zip(train_widths, anot_structures)]
an_max_y_diff = [[get_diff_y(coords)/height for struct in anot_structure for coords in struct['geometry']['coordinates']] 
     for height, anot_structure in zip(train_heights, anot_structures)]
an_area = [[100*np.pi*x*y/4 for x, y in zip( x_img, y_img)] for x_img, y_img in zip(an_max_x_diff, an_max_y_diff)] 

In [ ]:
plt.title("Total Area Filled with Anatomical Structure | Train ", fontsize=30)
total_areas = [np.sum(an) for an in an_area]
plt.bar(train_imgs, total_areas)
plt.ylabel("Area, %", fontsize=24)
plt.text(-3, int(np.max(total_areas)), f"Mean area {round(np.mean(total_areas), 2)}%", fontsize=20);

### Fantastic shapely visualization

In [ ]:
pols_markup = [[Polygon(glom['geometry']['coordinates'][0]) for glom in markup] 
     for markup in markups]

In [ ]:
def convert_multi_into_plain(coords: list):
    print(coords)
    list_coord = []
    for coord in coords:
        list_coord.append(np.array(coord[0]))
    return np.concatenate(list_coord, axis=0).tolist()

In [ ]:
%%capture
for i in range(len(anot_structures)):
    for j in range(len(anot_structures[i])):
        if anot_structures[i][j]['geometry']['type'] == 'MultiPolygon':
            anot_structures[i][j]['geometry']['coordinates'] = [convert_multi_into_plain(anot_structures[i][j]['geometry']['coordinates'])] 

In [ ]:
pols_anot = [[Polygon(struct['geometry']['coordinates'][0]) for struct in anot_structure] 
           for anot_structure in anot_structures]

In [ ]:
pols_anot_name = [[struct['properties']['classification']['name'] for struct in anot_structure] 
           for anot_structure in anot_structures]

In [ ]:
import shapely.ops as so
import matplotlib.pyplot as plt
import shapely.geometry as sg
import matplotlib.patches as mpatches

blue_patch = mpatches.Patch(color='blue', label='Medulla')
green_patch = mpatches.Patch(color='green', label='Cortex')


num_cols = 4
length = len(pols_markup)
num_rows = length // num_cols
plt.rcParams['figure.figsize'] = [16, 12]
fig, ax = plt.subplots(num_rows, num_cols, sharex=False, sharey=False)
fig.suptitle("Localization of Glomerulus", fontsize=24)
fig.text(0.5, 0.04, "Width", ha='center' ,fontsize=24)
fig.text(-0.02, 0.5, "Height", va='center', rotation='vertical', fontsize=24)
fig.tight_layout(pad=3.0)
    
for i in range(length):
    for geom in pols_markup[i]:
        xs, ys = geom.exterior.xy    
        ax[i//num_cols, i%num_cols].fill(xs, ys, alpha=0.5, fc='r', ec='none')
        if i%num_cols == num_cols-1:
            ax[i//num_cols, i%num_cols].legend(handles=[blue_patch, green_patch], bbox_to_anchor=(1.5, 1.0), loc='upper right')
    
    for colour, geom in zip(pols_anot_name[i], pols_anot[i]): 
        xs, ys = geom.exterior.xy    
        if colour == 'Medulla':
            ax[i//num_cols, i%num_cols].fill(xs, ys, alpha=0.25, fc='b', ec='none')
        elif colour == 'Cortex':
            ax[i//num_cols, i%num_cols].fill(xs, ys, alpha=0.25, fc='g', ec='none')
        else:
            print("Error")
    ax[i//num_cols, i%num_cols].set(aspect=1.0)
    ax[i//num_cols, i%num_cols].set_title(train_imgs[i], fontsize=14)
plt.savefig('data/plots/glom_localization.pdf', bbox_inches='tight') 

In [ ]:
from tqdm import tqdm
list_med,  list_cort, list_rest = [], [] , []
for i in tqdm(range(length)):
    med, cort, rest, glom  = 0, 0, 0, 0
    for geom in pols_markup[i]:
        glom += geom.area
        for colour, anot in zip(pols_anot_name[i], pols_anot[i]):
            area = anot.intersection(geom).area
            if colour == 'Medulla':
                med += area
            elif colour == 'Cortex':
                cort += area
            else:
                print("Error")
            rest += area
    list_med.append(round(med, 2)), list_cort.append(round(cort, 2))
    list_rest.append(round(glom - rest, 2))

In [ ]:
for i in range(8):
    total = list_cort[i] + list_med[i] + list_rest[i]
    cort_proc = round(list_cort[i]*100/total, 1)
    med_proc = round(list_med[i]*100/total, 1)
    rest_proc = round(list_rest[i]*100/total, 1)
    print(f"{train_imgs[i]} Itersection glomerulus with Cortex, Medulla, None: {list_cort[i]} ({cort_proc}%), {list_med[i]} ({med_proc}%), {list_rest[i]} ({rest_proc}%)")

In [ ]:
def norm(polyg):
    return (np.array(polyg) - np.mean(polyg, axis=0)).tolist()

In [ ]:
markups_norm = [[norm(glom['geometry']['coordinates'][0]) for glom in markup] 
     for markup in markups]

In [ ]:
markups_norm_max = [[list(np.max(abs(np.array(glom)), axis=0).astype(int)+1) for glom in markup] 
                    for markup in markups_norm]
markups_norm_max = [list(np.max(markup, axis=0)) for markup in markups_norm_max]

In [ ]:
def pol_to_mask(poly_verts, max_val):
    
    nx, ny = max_val[0], max_val[1]
    x, y = np.meshgrid(np.arange(-nx+1, nx+1), np.arange(-ny+1, ny+1))
    x, y = x.flatten(), y.flatten()

    points = np.vstack((x,y)).T

    path = Path(poly_verts)
    grid = path.contains_points(points)
    grid = grid.reshape((2*ny,2*nx))
    return np.array(grid, dtype=int)

In [ ]:
markups_norm_mask = [[pol_to_mask(glom, max_val) for glom in markup] 
                     for max_val, markup in zip(markups_norm_max, markups_norm)]

In [ ]:
markups_norm_mask_sum=[np.sum(np.array(masks), axis=0) for masks in markups_norm_mask]

In [ ]:
def create_ticks(values, step=50):
    r_x =list(np.arange(values[0]+1+step, values[0]*2, step))
    l_x = sorted(list(np.arange(values[0]+1, 0, -step)))
    
    r_y =list(np.arange(values[1]+1+step, values[1]*2, step))
    l_y = sorted(list(np.arange(values[1]+1, 0, -step)))
    
    new_x = [el - values[0] - 1 for el in l_x+r_x]
    new_y = [el - values[1] - 1 for el in l_y+r_y]
    return l_x+r_x, l_y + r_y, new_x, new_y

In [ ]:
num_cols = 4
length = len(pols_markup)
num_rows = length // num_cols
plt.rcParams['figure.figsize'] = [20, 14]
fig, ax = plt.subplots(num_rows, num_cols)
fig.suptitle("Shape of Glomerulus", fontsize=24)
fig.text(0.5, 0.04, "Width", ha='center' ,fontsize=24)
fig.text(-0.02, 0.5, "Height", va='center', rotation='vertical', fontsize=24)
fig.tight_layout(pad=6.0)

for i in tqdm(range(length)):
    sns.heatmap(markups_norm_mask_sum[i], ax=ax[i//num_cols, i%num_cols])
    ax[i//num_cols, i%num_cols].set_title(train_imgs[i],fontsize=28)
    ax[i//num_cols, i%num_cols].set(aspect=1.0)
    x, y, nx, ny = create_ticks(markups_norm_max[i], step=50)
    ax[i//num_cols, i%num_cols].set_xticks(x)
    ax[i//num_cols, i%num_cols].set_xticklabels(nx)
    ax[i//num_cols, i%num_cols].set_yticks(y)
    ax[i//num_cols, i%num_cols].set_yticklabels(ny)
plt.savefig('data/plots/glom_shape.pdf', bbox_inches='tight') 